# Collecting Amazon Feedback for Smartphone Sentiment Analysis

## Introduction

In today's digital era, customer reviews play a crucial role in shaping purchase decisions, especially in the highly competitive smartphone market. This project focuses on scraping customer reviews from Amazon for three flagship smartphones: Apple iPhone 15, Samsung Galaxy S24, and Google Pixel 7 5G.

The goal is to collect real user feedback directly from product pages on Amazon using web scraping techniques. These reviews provide valuable insights into customer satisfaction, common issues, and feature preferences. The collected data serves as the foundation for sentiment analysis and comparative studies between the devices, helping to understand public perception and user experience.

This process includes:

- Identifying and extracting review content (titles, bodies, ratings, etc.)

- Storing the data in structured format (CSV)

- Preprocessing for natural language analysis

Due to Amazon's security and access limitations, scraping was performed cautiously and ethically, respecting rate limits and avoiding authentication-required content.

### Import Libraries

In [1]:
#Import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

### Amazon Review Scraper Function with Selenium and BeautifulSoup

In [4]:
# Function: Scrape Amazon Product Reviews Using Selenium (Manual Login)
def scrape_amazon_reviews(product_url, num_pages, output_csv):
    """
    Scrapes Amazon reviews for the given product URL and saves them to a CSV file.
    Includes a manual login step to handle restricted review pages.

    Args:
        product_url (str): The Amazon product review page URL.
        num_pages (int): Number of pages to scrape.
        output_csv (str): Output filename for the CSV.

    Returns:
        pd.DataFrame: DataFrame containing the scraped reviews.
    """

    # Step 1: Launch browser and open login page
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.amazon.in/ap/signin")

    # Step 2: Wait for manual login
    input("🔐 Please log in to Amazon in the browser window, then press Enter to continue...")

    # Step 3: Load the review page
    driver.get(product_url)
    time.sleep(3)  # Give time for page to load

    reviews_data = []

    for page in range(1, num_pages + 1):
        print(f"🔄 Scraping Page {page}...")

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[data-hook='review-title']"))
            )
        except:
            print("⚠️ Reviews not found on page. Maybe still blocked or redirected.")
            break

        soup = BeautifulSoup(driver.page_source, "html.parser")

        titles = soup.select("a[data-hook='review-title']")
        ratings = soup.select("i[data-hook='review-star-rating'], i[data-hook='cmps-review-star-rating']")
        bodies = soup.select("span[data-hook='review-body']")
        dates = soup.select("span[data-hook='review-date']")

        for t, r, b, d in zip(titles, ratings, bodies, dates):
            reviews_data.append({
                "Title": t.get_text(strip=True),
                "Rating": r.get_text(strip=True).split()[0],
                "Review": b.get_text(strip=True),
                "Date": d.get_text(strip=True)
            })

        # Try clicking the "Next" button
        try:
            next_button = driver.find_element(By.XPATH, "//li[@class='a-last']/a")
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)
        except:
            print("❌ No more pages or 'Next' button not found.")
            break

    driver.quit()

    # Save to CSV
    df = pd.DataFrame(reviews_data)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Scraped {len(df)} reviews → Saved to '{output_csv}'")
    return df

### Scraping Amazon Reviews for iPhone 15

In [11]:
#Url of Iphone 15 amazon reviews
iphone_url = "https://www.amazon.in/product-reviews/B0CHX1W1XY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&"

In [13]:
#Scraping Iphone 15 amazon reviews
scrape_amazon_reviews(iphone_url, 10, "iphone15_amazon_reviews.csv")

🔐 Please log in to Amazon in the browser window, then press Enter to continue... 


🔄 Scraping Page 1...
🔄 Scraping Page 2...
🔄 Scraping Page 3...
🔄 Scraping Page 4...
🔄 Scraping Page 5...
🔄 Scraping Page 6...
🔄 Scraping Page 7...
🔄 Scraping Page 8...
🔄 Scraping Page 9...
🔄 Scraping Page 10...
❌ No more pages or 'Next' button not found.

✅ Scraped 100 reviews → Saved to 'iphone15_amazon_reviews.csv'


,Title,Rating,Review,Date
0,5.0 out of 5 starsiPhone XR to iPhone 15,5.0,Just made the transition and here’s my POV on ...,Reviewed in India on 7 October 2023
1,5.0 out of 5 starsStunning Pink – A Perfect Bl...,5.0,I recently purchased the iPhone 15 in Pink fro...,Reviewed in India on 21 March 2025
2,5.0 out of 5 starsiPhone 15 A solid upgrade an...,5.0,DesignOne of the most noticeable changes to th...,Reviewed in India on 6 October 2023
3,"4.0 out of 5 stars⭐️⭐️⭐️⭐️⭐️ Sleek, Fast & Rel...",4.0,The iPhone 15 is a noticeable step up from pre...,Reviewed in India on 26 May 2025
4,5.0 out of 5 starsProduct good delevery bad,5.0,Product was good but the delivery boy from blu...,Reviewed in India on 17 June 2025
...,...,...,...,...
95,5.0 out of 5 starsGood,5.0,Good,Reviewed in India on 16 May 2025
96,5.0 out of 5 starsHmm..,5.0,Same as shown but features was not so good as ...,Reviewed in India on 21 April 2025
97,5.0 out of 5 starsSecy,5.0,Secy,Reviewed in India on 15 May 2025
98,5.0 out of 5 starsBest phone,5.0,Best phone,Reviewed in India on 28 April 2025


### Scraping Amazon Reviews for Samsung S24

In [8]:
#Url of samsung s24 amazon reviews
samsung_url = "https://www.amazon.in/product-reviews/B0CS69DGSW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

In [10]:
#Scraping Samsung s24 amzon Reviews
scrape_amazon_reviews(samsung_url, 10, "S24_amazon_reviews.csv")

🔐 Please log in to Amazon in the browser window, then press Enter to continue... 


🔄 Scraping Page 1...
🔄 Scraping Page 2...
🔄 Scraping Page 3...
🔄 Scraping Page 4...
🔄 Scraping Page 5...
🔄 Scraping Page 6...
🔄 Scraping Page 7...
🔄 Scraping Page 8...
🔄 Scraping Page 9...
🔄 Scraping Page 10...
❌ No more pages or 'Next' button not found.

✅ Scraped 100 reviews → Saved to 'S24_amazon_reviews.csv'


,Title,Rating,Review,Date
0,"5.0 out of 5 starsNot flagship material,..but ...",5.0,This is an updated review :So just moved from ...,Reviewed in India on 17 May 2024
1,5.0 out of 5 starsAn amazing compact phone - T...,5.0,I have been using this phone for a month now a...,Reviewed in India on 10 March 2024
2,4.0 out of 5 starsMy true feelings about this ...,4.0,Everything is on point as expected. But there ...,Reviewed in India on 9 June 2025
3,5.0 out of 5 starsgood purchase,5.0,"Small and fast, perfect!",Reviewed in India on 30 June 2025
4,1.0 out of 5 starsDo not buy,1.0,"The model doesnt feel upgraded in any manner, ...",Reviewed in India on 30 June 2025
...,...,...,...,...
95,3.0 out of 5 starsGood display and performance,3.0,Battery backup is not good.Minor heating issue...,Reviewed in India on 11 October 2024
96,2.0 out of 5 starsI am not happy with S24 piece.,2.0,During video calls the phone gets heated up.Fi...,Reviewed in India on 2 February 2025
97,5.0 out of 5 starsAmazing Product,5.0,The media could not be loaded.The product sams...,Reviewed in India on 10 January 2025
98,3.0 out of 5 starsPls don't buy,3.0,Heat issues and battery life is very bad,Reviewed in India on 26 October 2024


### Scraping Amazon Reviews for Google Pixel7 5g

In [12]:
#Url of google pixel7 5g amazon reviews
pixel_url ="https://www.amazon.in/product-reviews/B0BJL8H3QY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

In [14]:
#Scaraping amzon reviews of google pixel7 5g
scrape_amazon_reviews(pixel_url, 10, "pixel_amazon_reviews.csv")

🔐 Please log in to Amazon in the browser window, then press Enter to continue... 


🔄 Scraping Page 1...
🔄 Scraping Page 2...
🔄 Scraping Page 3...
🔄 Scraping Page 4...
🔄 Scraping Page 5...
🔄 Scraping Page 6...
🔄 Scraping Page 7...
🔄 Scraping Page 8...
🔄 Scraping Page 9...
🔄 Scraping Page 10...
❌ No more pages or 'Next' button not found.

✅ Scraped 100 reviews → Saved to 'pixel_amazon_reviews.csv'


,Title,Rating,Review,Date
0,"5.0 out of 5 starsIf you want Android phone, d...",5.0,After using it for a year.The Pixel 7 is a fan...,Reviewed in India on 28 September 2024
1,1.0 out of 5 starsThe worst mobike you could b...,1.0,This is japanese model not induan version. The...,Reviewed in India on 25 June 2023
2,4.0 out of 5 starsGreat feature packed phone a...,4.0,Got the phone at 45k with credit card discount...,Reviewed in India on 22 June 2023
3,5.0 out of 5 starsPixel in a budget. (A review...,5.0,If you have pushed your budget till 50k you mi...,Reviewed in India on 12 May 2023
4,5.0 out of 5 starsAmazing product,5.0,The phone itself is a masterpiece the AI makes...,Reviewed in India on 11 January 2023
...,...,...,...,...
95,1.0 out of 5 starsHeating,1.0,Don't buy this product in Amazon Google suppor...,Reviewed in India on 10 February 2023
96,1.0 out of 5 starsModel,1.0,I am not happy with pixel 7,Reviewed in India on 7 May 2023
97,1.0 out of 5 starsRecieved a second hand phone.,1.0,Someone else has used and has done the reset i...,Reviewed in India on 31 January 2023
98,1.0 out of 5 starsnot used but phone heating p...,1.0,Phone heating is very much,Reviewed in India on 1 June 2023


## Conclusion

A total of 100 customer reviews were successfully scraped from Amazon for each of the three smartphones: iPhone 15, Samsung Galaxy S24, and Google Pixel 7 5G. These reviews include detailed user feedback, star ratings, and sentiment-rich content that will be instrumental in performing further sentiment analysis and comparative evaluation.

Despite Amazon's increasing restrictions on data access, the scraping process was executed efficiently using automated tools, while maintaining ethical practices and avoiding login-restricted content. The collected data now forms a robust foundation for understanding real customer experiences and opinions, enabling deeper analysis in the next stages of the project.